In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
import mysql.connector
from IPython.display import clear_output

In [2]:
data = pd.read_csv('combined_data.csv')
data['Rainfall.mm.'] = pd.to_numeric(data['Rainfall.mm.'], errors='coerce')  # Ensure Rainfall is numeric
data = data.dropna(subset=['Rainfall.mm.'])
data.head()


# Features and Target
X = data[['Initial_Year', 'Initial_Month', 'Forecast_Year', 'Foreast_Month', 'Province_ID']]
y = data['Rainfall.mm.']

# Scale the feature data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)



model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  
])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
optimizer = Adam(learning_rate=0.0001, clipvalue=1.0)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])



c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training parameters
roun = 10  # Number of epochs
batch_siz = 32  # Batch size
train_round = 3  # Number of training iterations

rain_probabilities = []

for i in range(train_round):
    print("--------------------------------------------------")
    print(f"Training iteration {i+1}/{train_round}")
    print("--------------------------------------------------")
    
    model.fit(X_scaled, y, epochs=roun, batch_size=batch_siz, verbose=0)
    model.save("model.h5")  # Save the model after training

    # Prediction on the last day in the dataset
    probability_last_day = model.predict(X_scaled[-1].reshape(1, -1))
    rain_probabilities.append(probability_last_day[0][0] * 100)

average_probability = sum(rain_probabilities) / len(rain_probabilities)
print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print("----------------------------------------------------------------")
print(f"ค่าเฉลี่ยความน่าจะเป็นในการเกิดฝนหลังจาก {train_round} รอบ: {average_probability:.2f}%")
print("================================================================")

# Testing section
print("================================TESTING===============================")
provinces = [
    (1, "กรุงเทพมหานคร"),
    (2, "จังหวัดกาญจนบุรี"),
    (3, "จังหวัดการบุรีรัมย์"),
    (4, "จังหวัดขอนแก่น"),
    (5, "จังหวัดจันทบุรี"),
    (6, "จังหวัดชลบุรี"),
    (7, "จังหวัดชุมพร"),
    (8, "จังหวัดเชียงราย"),
    (9, "จังหวัดเชียงใหม่"),
    (10, "จังหวัดตรัง"),
    (11, "จังหวัดตราด"),
    (12, "จังหวัดนครนายก"),
    (13, "จังหวัดนครปฐม"),
    (14, "จังหวัดนครราชสีมา"),
    (15, "จังหวัดนครศรีธรรมราช"),
    (16, "จังหวัดน่าน"),
    (17, "จังหวัดบุรีรัมย์"),
    (18, "จังหวัดปทุมธานี"),
    (19, "จังหวัดพะเยา"),
    (20, "จังหวัดพิษณุโลก"),
    (21, "จังหวัดเพชรบุรี"),
    (22, "จังหวัดเพชรบูรณ์"),
    (23, "จังหวัดแม่ฮ่องสอน"),
    (24, "จังหวัดลำปาง"),
    (25, "จังหวัดลำพูน"),
    (26, "จังหวัดเลย"),
    (27, "จังหวัดศรีสะเกษ"),
    (28, "จังหวัดสกลนคร"),
    (29, "จังหวัดสงขลา"),
    (30, "จังหวัดสุพรรณบุรี"),
    (31, "จังหวัดสุราษฎร์ธานี"),
    (32, "จังหวัดอำนาจเจริญ"),
    (33, "จังหวัดอุดรธานี"),
    (34, "จังหวัดอุตรดิตถ์"),
    (35, "จังหวัดเชียงราย"),
]

# Display provinces for user reference
for province_id, province_name in provinces:
    print(f"Province ID: {province_id}, Province Name: {province_name}")

# Sample test date
iD = 6  # Example day
iM = 9  # Example month
iY = 2024  # Example year
city_id = 9  # Example province ID

# Predict using the model
input_data = np.array([[iY, iM, iD, city_id]])  # Adjust as per feature requirements
input_data_scaled = scaler.transform(input_data)

result_probability = model.predict(input_data_scaled)
print("===========================OUTPUT===============================")
print(f"ความน่าจะเป็นในการเกิดฝน: {result_probability[0][0] * 100:.2f}%")
print("================================================================")

--------------------------------------------------
Training iteration 1/3
--------------------------------------------------


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
--------------------------------------------------
Training iteration 2/3
--------------------------------------------------


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
--------------------------------------------------
Training iteration 3/3
--------------------------------------------------


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
----------------------------------------------------------------
ค่าเฉลี่ยความน่าจะเป็นในการเกิดฝนหลังจาก 3 รอบ: 100.00%
================================TESTING===============================
Province ID: 1, Province Name: กรุงเทพมหานคร
Province ID: 2, Province Name: จังหวัดกาญจนบุรี
Province ID: 3, Province Name: จังหวัดการบุรีรัมย์
Province ID: 4, Province Name: จังหวัดขอนแก่น
Province ID: 5, Province Name: จังหวัดจันทบุรี
Province ID: 6, Province Name: จังหวัดชลบุรี
Province ID: 7, Province Name: จังหวัดชุมพร
Province ID: 8, Province Name: จังหวัดเชียงราย
Province ID: 9, Province Name: จังหวัดเชียงใหม่
Province ID: 10, Province Name: จังหวัดตรัง
Province ID: 11, Province Name: จังหวัดตราด
Province ID: 12, Province Name: จังหวัดนครนายก
Province ID: 13, Province Name: จังหวัดนครปฐม
Province ID: 14, Province Name: จังหวัดนครราชสีมา
Province ID: 15, Province Name: จังหวัดนครศรีธรรมราช
Province ID: 16, Province Name: จังหวั

ValueError: X has 4 features, but MinMaxScaler is expecting 5 features as input.